In [1]:
# Import libaries
import pandas as pd
from lxml import html, etree
import requests
import re
import os
import sys
import unicodecsv as csv
import argparse
import json
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request

In [4]:
def parse(keyword, place):

	headers = {	'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
				'accept-encoding': 'gzip, deflate, sdch, br',
				'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
				'referer': 'https://www.glassdoor.com/',
				'upgrade-insecure-requests': '1',
				'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
				'Cache-Control': 'no-cache',
				'Connection': 'keep-alive'
	}

	location_headers = {
		'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.01',
		'accept-encoding': 'gzip, deflate, sdch, br',
		'accept-language': 'en-GB,en-US;q=0.8,en;q=0.6',
		'referer': 'https://www.glassdoor.com/',
		'upgrade-insecure-requests': '1',
		'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36',
		'Cache-Control': 'no-cache',
		'Connection': 'keep-alive'
	}
	data = {"term": place,
			"maxLocationsToReturn": 10}

	location_url = "https://www.glassdoor.co.in/findPopularLocationAjax.htm?"
	try:
		# Getting location id for search location
		print("Fetching location details")
		location_response = requests.post(location_url, headers=location_headers, data=data).json()
		place_id = location_response[0]['locationId']
		job_listing_url = 'https://www.glassdoor.com/Job/jobs.htm'
		# Form data to get job results
		data = {
			'clickSource': 'searchBtn',
			'sc.keyword': keyword,
			'locT': 'C',
			'locId': place_id,
			'jobType': ''
		}

		job_listings = []
		if place_id:
			response = requests.post(job_listing_url, headers=headers, data=data)

			# extracting data from           
			# https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=true&clickSource=searchBtn&typedKeyword=andr&sc.keyword=android+developer&locT=C&locId=1146821&jobType=
			parser = html.fromstring(response.text)
			# Making absolute url 
			base_url = "https://www.glassdoor.com"
			parser.make_links_absolute(base_url)
            
			XPATH_ALL_JOB = '//li[@class="jl"]'
			XPATH_NAME = './/a/text()'
			XPATH_JOB_URL = './/a/@href'
			XPATH_LOC = './/span[@class="subtle loc"]/text()'
			XPATH_COMPANY = './/div[@class="flexbox empLoc"]/div/text()'
			XPATH_SALARY = './/span[@class="green small"]/text()'
			XPATH_DAYS_OLD = './/span[@class="minor"]/text()'
          

			listings = parser.xpath(XPATH_ALL_JOB)
			for job in listings:
				raw_job_name = job.xpath(XPATH_NAME)
				raw_job_url = job.xpath(XPATH_JOB_URL)
				raw_lob_loc = job.xpath(XPATH_LOC)
				raw_company = job.xpath(XPATH_COMPANY)
				raw_salary = job.xpath(XPATH_SALARY)
				raw_days_old = job.xpath(XPATH_DAYS_OLD)               

				# Cleaning data
				job_name = ''.join(raw_job_name).strip('–') if raw_job_name else None
				job_location = ''.join(raw_lob_loc) if raw_lob_loc else None
                
				raw_state = re.findall(",\s?(.*)\s?", job_location)
				state = ''.join(raw_state).strip()
                
				raw_city = job_location.replace(state, '')
				city = raw_city.replace(',', '').strip()
                
				company = ''.join(raw_company).replace('–','')
                
				salary = ''.join(raw_salary).strip()

				job_url = raw_job_url[0] if raw_job_url else None               
                
				days_old_1 = raw_days_old[0] if raw_days_old else None

				days_old_2 = raw_days_old[1] if raw_days_old else None                
    
				jobs = {
					"Name": job_name,
					"Company": company,
					"State": state,
					"City": city,
					"Salary": salary,
					"Location": job_location,
					"Days Old 1": days_old_1,
					"Days Old 2": days_old_2,                    
					"Url": job_url                  
				}
				job_listings.append(jobs)

			return job_listings
		else:
			print("location id not available")

	except:
		print("Failed to load locations")

if __name__ == "__main__":
    
	keyword = ['data scientist']
	place = ['boston']
   
	print("Fetching job details")

    
df = pd.DataFrame()
for k in keyword:
	for p in place:
		scraped_data = parse(k, p)
		print("Putting data into DataFrame")
		data = pd.DataFrame.from_dict(scraped_data)
		df = pd.concat([df, data], axis=0, ignore_index=True)      


Fetching job details
Fetching location details
Putting data into DataFrame


In [5]:
df.head()

,City,Company,Days Old 1,Days Old 2,Location,Name,Salary,State,Url
0,Boston,Wayfair,8 days ago,8d,"Boston, MA",Machine Learning Data Scientist,$96k-$137k,MA,https://www.glassdoor.com/partner/jobListing.h...
1,Boston,Press Ganey,23 days ago,23d,"Boston, MA",Data Scientist,$75k-$113k,MA,https://www.glassdoor.com/partner/jobListing.h...
2,Boston,Klaviyo New,None,None,"Boston, MA",Data Scientist,,MA,https://www.glassdoor.com/partner/jobListing.h...
3,Cambridge,H3 Biomedicine,None,None,"Cambridge, MA",Scientist/Sr Scientist – Invitro Oncology Drug...,,MA,https://www.glassdoor.com/partner/jobListing.h...
4,Framingham,Berg Health,None,None,"Framingham, MA",Data Scientist,$98k-$154k,MA,https://www.glassdoor.com/partner/jobListing.h...


In [6]:
# Job id column
job_ids = []
for i in range(len(df)):
    jid = df['Url'][i][-10:]
    job_ids.append(jid)

job_ids = pd.Series(job_ids)

# Add job_id column to main DataFrame
df = pd.concat([df, job_ids], axis=1)

# Rename job_id column
df.rename(columns={0: 'Job ID'}, inplace=True)

In [7]:
len(df)

30

In [8]:
df.head()

,City,Company,Days Old 1,Days Old 2,Location,Name,Salary,State,Url,Job ID
0,Boston,Wayfair,8 days ago,8d,"Boston, MA",Machine Learning Data Scientist,$96k-$137k,MA,https://www.glassdoor.com/partner/jobListing.h...,2908423996
1,Boston,Press Ganey,23 days ago,23d,"Boston, MA",Data Scientist,$75k-$113k,MA,https://www.glassdoor.com/partner/jobListing.h...,3008156171
2,Boston,Klaviyo New,None,None,"Boston, MA",Data Scientist,,MA,https://www.glassdoor.com/partner/jobListing.h...,2834799767
3,Cambridge,H3 Biomedicine,None,None,"Cambridge, MA",Scientist/Sr Scientist – Invitro Oncology Drug...,,MA,https://www.glassdoor.com/partner/jobListing.h...,3080400455
4,Framingham,Berg Health,None,None,"Framingham, MA",Data Scientist,$98k-$154k,MA,https://www.glassdoor.com/partner/jobListing.h...,3023734635


### Job URLs

In [9]:
job_urls = []
for i in range(len(df)):
    url = df['Url'][i]
    job_urls.append(url)

len(job_urls)

30

In [10]:
# Get industry & date posted
industries = []
dates = []

user_agent = 'linnea'
headers = {'User-Agent' : user_agent }

# For loop
for i in range(len(job_urls)):
    req = urllib.request.Request(job_urls[i], headers=headers)
    response = urllib.request.urlopen(req).read()
    
    industry = re.findall('"industry": \w*\ ?"\w*', str(response))
    date_posted = re.findall('"datePosted": "[0-9]+[-][0-9]+[-][0-9]+"', str(response))
    
    industries.append(industry)
    dates.append(date_posted)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [11]:
df2 = pd.DataFrame(columns=['URL_CHECK', 'Industry', 'Date Posted'])
   
df2['URL_CHECK'] = job_urls
df2['Industry'] = industries
df2['Date Posted'] = dates

In [12]:
df2['URL_CHECK'][3] == df['Url'][3]

True

In [13]:
df = pd.concat([df,df2], axis=1)

In [14]:
df.head()

,City,Company,Days Old 1,Days Old 2,Location,Name,Salary,State,Url,Job ID,URL_CHECK,Industry,Date Posted
0,Boston,Wayfair,8 days ago,8d,"Boston, MA",Machine Learning Data Scientist,$96k-$137k,MA,https://www.glassdoor.com/partner/jobListing.h...,2908423996,https://www.glassdoor.com/partner/jobListing.h...,"[""industry"": ""Retail]","[""datePosted"": ""2019-01-10""]"
1,Boston,Press Ganey,23 days ago,23d,"Boston, MA",Data Scientist,$75k-$113k,MA,https://www.glassdoor.com/partner/jobListing.h...,3008156171,https://www.glassdoor.com/partner/jobListing.h...,"[""industry"": ""Health]","[""datePosted"": ""2018-12-26""]"
2,Boston,Klaviyo New,None,None,"Boston, MA",Data Scientist,,MA,https://www.glassdoor.com/partner/jobListing.h...,2834799767,https://www.glassdoor.com/partner/jobListing.h...,"[""industry"": ""Information]","[""datePosted"": ""2019-01-15""]"
3,Cambridge,H3 Biomedicine,None,None,"Cambridge, MA",Scientist/Sr Scientist – Invitro Oncology Drug...,,MA,https://www.glassdoor.com/partner/jobListing.h...,3080400455,https://www.glassdoor.com/partner/jobListing.h...,"[""industry"": ""Health]","[""datePosted"": ""2019-01-08""]"
4,Framingham,Berg Health,None,None,"Framingham, MA",Data Scientist,$98k-$154k,MA,https://www.glassdoor.com/partner/jobListing.h...,3023734635,https://www.glassdoor.com/partner/jobListing.h...,"[""industry"": ""Biotech]","[""datePosted"": ""2019-01-08""]"


In [15]:
# df.to_csv('data_example')